In [0]:
import sympy as sp
import math
sp.init_printing(use_unicode=True)

In [0]:
def custom_latex_printer(exp, **options):
    from google.colab.output._publish import javascript
    url = "https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.3/latest.js?config=default"
    javascript(url=url)
    return sp.printing.latex(exp, **options)

sp.init_printing(use_latex="mathjax", latex_printer=custom_latex_printer)

# Course 4 Lab 8.2

We have sprint-mass-damper system as first order LTI system where the state of the system is described as the tangent vector:
$
x = \begin{bmatrix}
x(t)\\
\dot{x}(t)
\end{bmatrix}
$

As demonstrated in leture find the matrix $A$ and then compute the closed-form expression for 
$
f^t_{DHO}(x_0):=e^{At}x_0
$.

Then let $m=2$ kg, $b=3$ and the spring constant $k=2 N/m$. 

Let $x_0 = \begin{bmatrix}
4\\
0
\end{bmatrix}$. At $t=4$ 
what is the position of the mass? 

Remember? the position is the first row of the column vector $x=\begin{bmatrix}x(t)\\ \dot{x}(t)\end{bmatrix}$


Spring-mass-dumper system equation of motion

$f(t)=m\ddot{x}(t)+kx+b\dot{x}$

Now recast as $1^{st}$ order vector ODE

$\dot{x} = \begin{bmatrix}
v\\
\Xi_{DHO}(x)
\end{bmatrix} = 
\begin{bmatrix}
\dot{x}\\
-\frac{b}{m}\dot{x} - \frac{k}{m}x
\end{bmatrix}
=:f_{DHO}(x)=Ax;
A:=\begin{bmatrix}0 && 1\\
-\frac{k}{m} && -\frac{b}{m}\end{bmatrix}$

$\frac{d}{dt}e^{At} = Ae^{At}$


In [0]:
k,m,b,t = sp.symbols("k m b t")
A = sp.Matrix([[0,1],[-k/m, -b/m]]) #symbolic matrix 
vA = sp.Matrix([[0,1], [-1, 1.5]] ) # A matrux with substituted values
x0 = sp.Matrix([4, 0])

#sp.pprint(x0)

Now let diagonalize matrix $A$ to take exponential according formula
$A=E \Lambda E^{-1}$
where
$\Lambda =
\begin{bmatrix}
\lambda && 0\\
0 && \lambda
\end{bmatrix}$

In [33]:
P,D = vA.diagonalize()
display(D)
display(D[0,0])

⎡0.75 - 0.661437827766148⋅ⅈ              0             ⎤
⎢                                                      ⎥
⎣            0               0.75 + 0.661437827766148⋅ⅈ⎦

0.75 - 0.661437827766148⋅ⅈ

In [34]:
l=vA.eigenvals()
display(l)

⎧3   √7⋅ⅈ     3   √7⋅ⅈ   ⎫
⎨─ - ────: 1, ─ + ────: 1⎬
⎩4    4       4    4     ⎭

And now according formula
$e^{At} = 
E\begin{bmatrix}
e^{t\lambda_1} && 0\\
0 && e^{t\lambda_2}
\end{bmatrix}
E^{-1}$

In [40]:
display ( D[0,0] )

0.75 - 0.661437827766148⋅ⅈ

In [44]:
eL = sp.Matrix([ [sp.exp(4*D[0,0]).expand(complex=True), 0], [0, sp.exp(4*D[1,1]) ]  ])
display(eL)

⎡-17.6666102854123 - 9.55613282222995⋅ⅈ                   0                  ⎤
⎢                                                                            ⎥
⎢                                                          2.64575131106459⋅ⅈ⎥
⎣                  0                     20.0855369231877⋅ℯ                  ⎦

### Closed form solution
According formula
$\frac{d}{dt}e^{At} = Ae^{At}$ 

Integral or flow through some vector initial condition $x0$ is
$f^t_{DHO}(x0) = e^{At}x0$

In [41]:
X = (P*eL*P**-1)*x0
display(complex(X[1]))

(-57.790059298565176+4.235164736271502e-22j)

In [0]:
E_s = sp.Matrix.ones(2,2)
E_s[:,0] =  sp.Matrix( [P[:,0]/P[1,0]] )
E_s[:,1] = sp.Matrix([P[:,1]/P[1,1] ])
#E_s = E_s.col_insert(0, s_1) #  ([ (), (P[:,1]/P[1,1]) ])
E_s = E_s.subs([(k, 2), (m,2), (b,3), (t,4)])
display(E_s)

⎡  -4        -4    ⎤
⎢────────  ────────⎥
⎢3 + √7⋅ⅈ  3 - √7⋅ⅈ⎥
⎢                  ⎥
⎣   1         1    ⎦

In [0]:
v = sp.Matrix([2,-2])
Z = E_s.T *v
display(Z )

⎡        8    ⎤
⎢-2 - ────────⎥
⎢     3 + √7⋅ⅈ⎥
⎢             ⎥
⎢        8    ⎥
⎢-2 - ────────⎥
⎣     3 - √7⋅ⅈ⎦

In [0]:
temp_v = sp.Matrix([ [1, sp.I], [1, -sp.I] ])
S = E_s * temp_v
display(S) 

⎡     4          4          4⋅ⅈ        4⋅ⅈ   ⎤
⎢- ──────── - ────────  - ──────── + ────────⎥
⎢  3 - √7⋅ⅈ   3 + √7⋅ⅈ    3 + √7⋅ⅈ   3 - √7⋅ⅈ⎥
⎢                                            ⎥
⎣          2                      0          ⎦

In [0]:
vA = A.subs([(k, 2), (m,2), (b,3), (t,4)])
A_tilde = S.inv()*vA*S
display(A_tilde)

⎡                                3      2          2                          
⎢                              - ─ + ──────── + ────────                      
⎢                                2   3 + √7⋅ⅈ   3 - √7⋅ⅈ                      
⎢                                                                             
⎢            ⎛     4          4    ⎞ ⎛   4          4    ⎞        ⎛   4       
⎢         √7⋅⎜- ──────── - ────────⎟⋅⎜──────── + ────────⎟   3⋅√7⋅⎜──────── + 
⎢  4⋅√7      ⎝  3 - √7⋅ⅈ   3 + √7⋅ⅈ⎠ ⎝3 + √7⋅ⅈ   3 - √7⋅ⅈ⎠        ⎝3 + √7⋅ⅈ   
⎢- ──── + ──────────────────────────────────────────────── + ─────────────────
⎣   7                            7                                       7    

                            2⋅ⅈ        2⋅ⅈ                 ⎤
                        - ──────── + ────────              ⎥
                          3 - √7⋅ⅈ   3 + √7⋅ⅈ              ⎥
                                                           ⎥
   4    ⎞     ⎛    4⋅ⅈ        4⋅ⅈ   ⎞ ⎛   4 